In [7]:
import datetime
import json
import pandas as pd
import requests
import sqlalchemy
import pandas.io.sql as sqlio 
import psycopg2 as ps 

f = open("/Users/katharinatielking/creds/credentials.json")
credentials = json.load(f)

file_path = list(credentials.values())[0]
weather_key = list(credentials.values())[1]
aws_key = list(credentials.values())[2]
aws_access_key = list(credentials.values())[3]
aws_secret_key = list(credentials.values())[4]
rds_host = list(credentials.values())[5]
rds_user = list(credentials.values())[6]
rds_password = list(credentials.values())[7]
rds_database = list(credentials.values())[8]
rds_charset = list(credentials.values())[9]
pw_postgres = list(credentials.values())[10]

In [8]:
# Connecting to AWS PostgreSQL DB
try:
    conn2 = ps.connect(dbname="postgres", 
                       user = "postgres", 
                       password = pw_postgres, 
                       host = "current-weather-data.cdg2kkaye7cw.us-east-1.rds.amazonaws.com", 
                       port = "5432")
    conn2.autocommit = True
    cur = conn2.cursor()
    print('Connection established.')
except:
    print("Error while connecting to PostgreSQL.")

Connection established.


In [20]:
# Request current weather data
location = pd.DataFrame({'city-name': ['Berlin', 'Tulum', 'Ko Tao', 'Parque Nacional Corvocado', 'San Diego'], 'latitude': [52.520007, 20.211419, 10.095610, 8.540835, 32.715738 ] , 'longitude': [13.404954, -87.465350, 99.840396, -83.570964, -117.161084]})

openweathermap_api_key = weather_key

def current_weather(row):
    url = f"https://api.openweathermap.org/data/2.5/weather?lat={row.latitude}&lon={row.longitude}&appid={openweathermap_api_key}"
    response = requests.get(url)
    response_json = response.json()
    sunset_utc = datetime.datetime.fromtimestamp(response_json["sys"]["sunset"])
    return {
        "main.temp": response_json["main"]["temp"] - 273.15,
        "rain.1h": response_json.get("rain", {}).get("1h", 0.0)
    }

df_weather = location.apply(lambda x: current_weather(x), axis=1)
current_weather = pd.concat([location, pd.json_normalize(df_weather)], axis=1)
print(current_weather)

##### TODO: ADD Y, YM, TIME STAMP


                   city-name   latitude   longitude  main.temp  rain.1h
0                     Berlin  52.520007   13.404954       2.09      0.0
1                      Tulum  20.211419  -87.465350      28.76      0.0
2                     Ko Tao  10.095610   99.840396      25.65      0.0
3  Parque Nacional Corvocado   8.540835  -83.570964      28.47      0.0
4                  San Diego  32.715738 -117.161084      16.54      0.0


In [ ]:
# Transform dataframe to the same format as the weather 

In [11]:
# Create RDS table for current weather
try:
    cur.execute("CREATE TABLE IF NOT EXISTS weather_current (city VARCHAR(255), country VARCHAR(255), latitude float8, longitude float8, temp float8)")
    print("Table created successfully")
except:
    print("Error while creating table")

try:
    for i, row in current_weather.iterrows():
        cur.execute("INSERT INTO weather_current (city, country, latitude, longitude, temp) VALUES (%s, %s, %s, %s, %s)", (
            row['city'], row['country'], row['latitude'], row['longitude'], row['temp']))
        print("Data inserted successfully")
except:
    print("Error while inserting data into table")


conn2.commit()    
cur.close()
conn2.close()

Table created successfully
Data inserted successfully
Data inserted successfully
Data inserted successfully
Data inserted successfully
Data inserted successfully


In [1]:
!pip freeze

aiohttp==3.9.1
aiosignal==1.3.1
aiosqlite==0.19.0
alembic==1.13.1
alpha-vantage==2.3.1
annotated-types==0.6.0
anyio==3.7.1
appdirs==1.4.4
appnope==0.1.3
apprise==1.6.0
argon2-cffi==23.1.0
argon2-cffi-bindings==21.2.0
arrow==1.3.0
asgi-lifespan==2.1.0
asttokens==2.4.1
async-lru==2.0.4
async-timeout==4.0.3
asyncpg==0.29.0
attrs==23.1.0
Babel==2.14.0
beautifulsoup4==4.12.2
bleach==6.1.0
boto==2.49.0
boto3==1.34.10
botocore==1.34.10
bs4==0.0.1
cachetools==5.3.2
caugetch==0.0.1
certifi==2023.11.17
cffi==1.16.0
charset-normalizer==3.3.2
click==8.1.7
clipboard==0.0.4
cloudpickle==3.0.0
colorama==0.4.6
comm==0.2.0
contourpy==1.0.6
coolname==2.2.0
croniter==2.0.1
cryptography==41.0.7
cssselect==1.2.0
cycler==0.11.0
dateparser==1.2.0
DateTime==5.4
debugpy==1.8.0
decorator==5.1.1
defusedxml==0.7.1
dnspython==2.4.2
docker==6.1.3
email-validator==2.1.0.post1
exceptiongroup==1.2.0
executing==2.0.1
fake-useragent==1.4.0
fastjsonschema==2.19.0
feedparser==6.0.11
finvizfinance==0.14.7
fonttools==4.38.0